
BERT en un problema de clasificación
====================================

Introducción
------------

Los modelos basados en transformers nos pueden ayudar a resolver varios tipos de problemas. Desde problemas de clasificación y regresión hasta tareas más complejas como resumen de textos o generación de leguaje condicionado. Veamos como resolver el problema de clasificación de tweets sobre el que hemos estado trabajando anteriormente pero ahora utilizando el modelo BERT.

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [1]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv \
    --quiet --no-clobber --directory-prefix ./Datasets/mascorpus/

!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextDataset.py \
    --quiet --no-clobber --directory-prefix ./Utils/
    
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/docs/nlp/neural/BERT.txt \
    --quiet --no-clobber
!pip install -r BERT.txt --quiet

In [1]:
import warnings
warnings.filterwarnings('ignore')

Cargamos el set de datos

In [2]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'], 
                                                    test_size=0.33, 
                                                    stratify=tweets['SECTOR'])

### Verificando el hardware disponible

In [4]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print("Este notebook se está ejecutando en", device)

Este notebook se está ejecutando en cpu


Transferencia de Aprendizaje y Fine-tuning
------------------------------------------

En general, existen 2 estrategias para utilizar modelos de lenguaje pre-entrenados en una tarea especifica:
 - Feature-based
 - Fine-tunning
 
Las técnicas que se conocen como **Feature-based** utiliza arquitecturas especificas para resolver cada una de las tareas de NLP, en donde los pesos de las representaciones vectoriales están "congeladas" y no son parámetros que el modelo deba optimizar. En consecuencia, estos modelos son más rápidos de entrenar y permiten aplicar arquitecturas especificas que sean diferenciales en cada una de las tareas.
 
Por el otro lado, las técnicas que emplean **Fine-tunning** tiene la flexibilidad de poder adaptar sus representaciones al permitir que todos los parametros sean optimizados en la tarea en particular. Además, estas arquitecturas permiten resolver multiples problemas de NLP utilizando una mínima cantidad de parametros específicos para la tarea.

## BETO: BERT en español

Al igual que con `word2vec`, entrenar un modelo de lenguaje requiere de una gran cantidad de datos sumado a un poder de computo interesante (cuando BERT fué publicado en 2018, tomó 4 días entrenar el modelo usando 16 TPUs. Si se hubiera entrenado en 8 GPUs hubiera tomado entre 40–70 días). Por este motivo, utilizaremos un modelo pre-entrenado para un cuerpo de texto en español. Este modelo, BETO, fué entrenado sobre un gran corpora de textos. Pueden encontrar más información sobre en [el sitio web del autor](https://github.com/dccuchile/beto).

### Tokenizers 

BERT utiliza su propio tokenizer que está basado en WordPiece. Este tokenizer tiene un vocabulario de 30.000 tokens donde cada secuencia comienza con un token especial [CLS]. Recuerden que los tokenizers dependen del modelo con el que estamos trabajando:

In [5]:
import transformers

tokenizer = transformers.BertTokenizerFast.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', 
                                                           do_lower_case=True)

*Noten que el tokenizer depende del modelo que estamos utilizando*

## Crando un modelo de clasificación basado en BERT

Trataremos de resolver entonces el mismo problema de clasificación con el que veniamos trabajando: clasificar los tweets dependiendo del sector al que pertenecen.Recordemos que tenemos 7 categorias distintas:

In [6]:
tweets['SECTOR'].unique()

array(['RETAIL', 'TELCO', 'ALIMENTACION', 'AUTOMOCION', 'BANCA',
       'BEBIDAS', 'DEPORTES'], dtype=object)

Necesitaremos contar con el numero de categorias para nuestro clasificador:

In [7]:
num_labels=len(tweets['SECTOR'].unique())

Antes de hacer fine-tunning de nuestro modelo, tenemos que instanciar el modelo sobre el cual queremos aplicar esta técnica. Para ello instanciaremos el modelo base el cual no está entrenado en ninguna tarea en particular. De hecho, si habilitan las alertas en este notebook, verán que cuando se carga el modelo, la libreria HuggingFace les advierte sobre esto:

In [8]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', 
                                                      num_labels=num_labels)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

Construiremos nuestro dataset sobre el que queremos entrenar el modelo. Recuerden que ya habíamos separado el set de datos en porciones para entrenar y para testear el modelo.

### Como entrenar modelos con Transformers

La librería transformers puede entrenar modelos tanto utilizando TensorFlow como PyTorch como backend. En nuestro caso utilizaremos PyTorch simplemente porque generaremos código un poco más compacto, pero pueden utilizar el backend con el que más cómodos se sientan:

### Set de datos

Para utilizar el objeto `Trainer` que provee `transformers`, necesitamos crear un objetivo de tipo `Dataset`. `PyTorch` implementa este objeto el modulo `torch.utils.data.Dataset`. Para simplificar la tarea, disponemos de una clase que hace todo el procesamiento de datos y generación de los conjuntos de datos utilizando dicho modulo. Pueden encontrar esta implementación en `Utils.ClassificationDataset`:

In [9]:
from Utils.TextDataset import ClassificationDataset

train_dataset = ClassificationDataset(examples=X_train, labels=y_train, tokenizer=tokenizer)
val_dataset = ClassificationDataset(examples=X_test, labels=y_test, tokenizer=tokenizer)

In [10]:
from transformers import Trainer, TrainingArguments

### Trainer

Especificamos los parametros con los que entrenaremos nuestro modelo:

In [11]:
training_args = TrainingArguments(
    output_dir='./results',          # Directorio de trabajo del Trainer
    num_train_epochs=3,              # Numero total de epochs sobre el que entrenaremos
    per_device_train_batch_size=16,  # Tamaño del batch de datos por cada dispositivo de entrenamiento
    per_device_eval_batch_size=64,   # Tamaño del batch de datos que usaremos para evaluación
    warmup_steps=500,                # Numero de pasos que se usaran para determinar la politica de Learning Rate
    weight_decay=0.01,               # Weight decay
    logging_dir='./logs',            # Directorio de logs
    logging_steps=10,
)

Instanciamos el Trainer

In [12]:
trainer = Trainer(
    model=model,                         # modelo sobre el que haremos fine tunning
    args=training_args,                  # parametros del entrenamiento
    train_dataset=train_dataset,         # set de datos de entrenamiento
    eval_dataset=val_dataset             # set de datos de evaluación
)

In [13]:
trainer.train()

***** Running training *****
  Num examples = 2521
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 474


Step,Training Loss
10,1.973900
20,1.959300
30,1.947800
40,1.866500
50,1.858700
60,1.844900
70,1.791900
80,1.695300
90,1.583200
100,1.398300




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=474, training_loss=0.50611606927565, metrics={'train_runtime': 3394.3375, 'train_samples_per_second': 2.228, 'train_steps_per_second': 0.14, 'total_flos': 314823941045550.0, 'train_loss': 0.50611606927565, 'epoch': 3.0})

Podemos inspeccionar nuestro modelo utilizando la herramienta `TensorBoard`

In [ ]:
!tensorboard --logdir ./logs --bind_all

En Google Colab, podemos ver `TensorBoard` usando el siguiente `Magic`

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./logs

Verifiquemos la performance de nuestro modelo

In [14]:
predictions = trainer.predict(test_dataset=val_dataset).predictions

***** Running Prediction *****
  Num examples = 1242
  Batch size = 64


Para evaluar el modelo, primero deberemos obtener cual es la categoria que obtuvo la mayor probabilidad en la clasificación:

In [15]:
import numpy as np

predictions = np.argmax(predictions, axis=1)

Convertimos los IDs de las categorias a los labels correctos

In [16]:
all_labels = val_dataset.get_labels()
predictions_label = [all_labels[idx] for idx in predictions]

In [17]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions_label))

              precision    recall  f1-score   support

ALIMENTACION       0.98      0.99      0.99       110
  AUTOMOCION       0.99      0.99      0.99       148
       BANCA       0.99      1.00      0.99       198
     BEBIDAS       1.00      1.00      1.00       223
    DEPORTES       0.98      0.98      0.98       216
      RETAIL       1.00      0.99      0.99       268
       TELCO       1.00      0.99      0.99        79

    accuracy                           0.99      1242
   macro avg       0.99      0.99      0.99      1242
weighted avg       0.99      0.99      0.99      1242



### Persistiendo el modelo

Podemos persistir el modelo para utilizar posteriormente de la siguiente forma. Note que deberemos persistir tanto el modelo como el `tokenizer` ya que los dos funcionan de la mano.

model_name = "tweet_classifier_bert"

trainer.save_model(model_name)
tokenizer.save_pretrained(model_name)

In [ ]:
!zip -r tweet_classification_bert.zip tweet_classification_bert